Connected to base (Python 3.11.7)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import xlwings as xl
import scipy.stats as stats
import os
import plotly.graph_objects as go
import datetime
from dateutil.relativedelta import relativedelta

os.chdir(r'P:/Amundi_Milan/Investment/SPECIALISTS/QUANT_RESEARCH/ASSET_ALLOCATION/crossasset/Jung/cbr/')
# os.chdir(r'/Users/jhkmm/Documents/python_mbpro/TAA-Report.git/TAA-Report')

In [2]:
data_file = 'cbr.xlsm'

data_macro = pd.read_excel(data_file,sheet_name='input-macro',header=0, index_col=0).dropna()
data_assets = pd.read_excel(data_file,sheet_name='input-assets',header=0, index_col=0).dropna()
cash = data_assets.iloc[:,-1]
cash_mom = cash.pct_change(periods=1).dropna()
# EQ
# data_assets = data_assets.iloc[:,0]
# GOV
# data_assets = data_assets.iloc[:,1]
# IG
# data_assets = data_assets.iloc[:,2]
# HY
# data_assets = data_assets.iloc[:,3]
# COMMO
# data_assets = data_assets.iloc[:,4]

delta_macro_yoy = data_macro.pct_change(periods=12).dropna()
delta_assets_yoy = data_assets.pct_change(periods=12).dropna()
delta_assets_mom = data_assets.pct_change(periods=1).dropna()
_,n_assets = data_assets.shape
n_assets -= 1

In [3]:
r_macro_y, c_macro_y = delta_macro_yoy.shape
df_pctile_macro = pd.DataFrame(index=delta_macro_yoy.index,columns=delta_macro_yoy.columns)
for j in range(c_macro_y):
    for i in range(r_macro_y):
        df_pctile_macro.iloc[i,j] = stats.percentileofscore(delta_macro_yoy.iloc[:i+1,j],delta_macro_yoy.iloc[i,j])/100

In [4]:
def calc_corr(macro_data: pd.DataFrame, asset_data: pd.DataFrame, asset_idx: int):  
    r1,c1 = macro_data.shape
    # print(asset_data.shape)
    r2, = asset_data.shape
    macro_data=macro_data[-min(r1,r2):]
    # TEMP Solution as we have one more data point for SPX
    asset_data=asset_data[-min(r1,r2):]
    # print(asset_data.shape)
    # print(macro_data.shape)
    corr_pearson = np.zeros((min(r1,r2)-35,c1))
    macro_val = macro_data.values        
    asset_val = asset_data.values
    # print(r2)
 
    for h in range(min(r1,r2)-35):
        # print(f"row{h}")
    
        for i in range(c1):
            # print(asset_val)
            # print(f"var{i}")
            # temp = macro_val[:h,i]
            # print(temp)

            corr_pearson[h,i] = stats.pearsonr(
                macro_val[:h+36,i],
                asset_val[:h+36]).correlation
            # print(corr_pearson[h,i])
    
    return pd.DataFrame(data=corr_pearson, index=asset_data.index[-(min(r1,r2)-35):], columns=macro_data.columns)

# corrs = calc_corr(macro_data=delta_macro_yoy,asset_data=delta_assets_mom.EQ, asset_idx=0)
# corr_gov = calc_corr(macro_data=delta_macro_yoy,asset_data=delta_assets_mom.GOV, asset_idx=0)
# significant_factors = np.abs(corrs) > 0.1
# r_corr,c_corr = corrs.shape
# factors = corrs.columns

In [6]:
corrs = pd.DataFrame()
for i in range(n_assets):
    corr_asset = calc_corr(macro_data=delta_macro_yoy,asset_data=data_assets.iloc[:,i], asset_idx=0)
    pd.concat([corrs, corr_asset])
    significant_factors = np.abs(corr_asset) > 0.1
    r_corr,c_corr = corr_asset.shape
    factors = corr_asset.columns

In [11]:
corr_asset2 = corr_asset
pd.concat([corr_asset, corr_asset2])

,Broad business,Recession alerts,Industrial Production,Manufacturing,Services,Small Business sentix,Consumption,Consumers sentiment,Labour general,Labour HH survey,...,Interest rates,Rates sentiment,CPI,CPI sentiment,PPI finished goods,BS_housing wealth,BS_housing refinancing,BS_current asset / liabilities,Savings rate,Fed liquidity
2005-03-31,-0.090577,-0.170661,-0.594394,0.031562,-0.281279,0.128727,-0.476517,0.039327,-0.376948,-0.492653,...,0.237939,-0.312558,0.612411,-0.284178,-0.287445,-0.113998,0.326132,-0.315247,-0.548394,-0.510724
2005-04-30,-0.109934,-0.212174,-0.589882,0.014460,-0.291524,0.127555,-0.475177,0.011805,-0.394612,-0.510860,...,0.214491,-0.267254,0.619959,-0.298496,-0.294520,-0.084022,0.333270,-0.172184,-0.533605,-0.494981
2005-05-31,-0.122371,-0.244614,-0.593738,0.011088,-0.301669,0.081551,-0.488131,-0.002230,-0.405730,-0.525249,...,0.178152,-0.245408,0.627598,-0.315075,-0.303261,-0.100383,0.312796,-0.089198,-0.530043,-0.488299
2005-06-30,-0.122357,-0.271329,-0.599570,0.037876,-0.290101,0.054922,-0.483884,-0.014186,-0.415246,-0.536261,...,0.139127,-0.247721,0.606136,-0.329855,-0.314753,-0.125122,0.281453,-0.111808,-0.532406,-0.494339
2005-07-31,-0.120409,-0.289032,-0.601031,0.053312,-0.289186,0.047295,-0.484336,-0.025113,-0.423857,-0.543932,...,0.121902,-0.265021,0.576156,-0.339351,-0.324443,-0.150181,0.248571,-0.133254,-0.537211,-0.487470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-30,-0.002969,0.037726,0.064363,0.031788,-0.036038,0.073550,0.072265,-0.047752,-0.065762,0.072742,...,0.081875,-0.115833,0.246393,-0.056190,-0.015666,0.024548,0.287601,-0.082461,0.090687,-0.023982
2024-07-31,-0.006723,0.042362,0.063771,0.029867,-0.034990,0.070757,0.071734,-0.050465,-0.068379,0.070809,...,0.076451,-0.114596,0.242848,-0.053796,-0.014736,0.021493,0.283694,-0.082983,0.087824,-0.028488
2024-08-31,-0.010838,0.046462,0.063104,0.025402,-0.035042,0.068166,0.071158,-0.052179,-0.070205,0.068786,...,0.069078,-0.117647,0.238183,-0.050833,-0.013526,0.018332,0.279386,-0.083287,0.084644,-0.031419
2024-09-30,-0.013265,0.048606,0.062203,0.022813,-0.032779,0.065346,0.070531,-0.053118,-0.071739,0.066866,...,0.059991,-0.111891,0.231764,-0.050371,-0.015553,0.015320,0.272343,-0.087889,0.081254,-0.033245


In [ ]:
def calc_dist(corrs: pd.DataFrame):
    low = []; high = []
    significant_factors_names = []
    desc_low = pd.DataFrame(); desc_high = pd.DataFrame()
    for i in range(r_corr):
        # point in time split
        df_pctile_macro_subset = df_pctile_macro[df_pctile_macro.index<=corrs.index[i]]
        # choose only significant factors (both macro and correlations)
        significant_factors_names.append(factors[significant_factors.iloc[i]].tolist())
        df_pctile_macro_subset = df_pctile_macro_subset.iloc[:,significant_factors.values[i,:]]
        corr_subset = corrs.iloc[i,significant_factors.values[i,:]]
        # multiply 
        corrs_pct_df = pd.DataFrame(data=np.multiply(df_pctile_macro_subset.values, np.abs(corr_subset.values.T)), 
                                        index=df_pctile_macro_subset.index,
                                        columns=df_pctile_macro_subset.columns)
        # split ref and comparison pts
        ref_df_pctile_macro_subset = corrs_pct_df.iloc[-1,:] 
        hist_df_pctile_macro_subset = corrs_pct_df.iloc[:-1,:]
        # calcuate distances 
        distances_full_subset = hist_df_pctile_macro_subset - ref_df_pctile_macro_subset 
        # TODO: np.linalg.norm is in a for loop. Getting the following error when trying to calculate norm for a matrix
        # TODO: TypeError: loop of ufunc does not support argument 0 of type numpy.float64 which has no callable sqrt method
        r,c = distances_full_subset.shape
        distances_norm = []
        for row in range(r):
            distances_norm.append(np.linalg.norm(distances_full_subset.iloc[row]))
        distances_norm = pd.DataFrame(distances_norm, columns = ["L2 Norm"], index = distances_full_subset.index)
        # locate points outside of the bounds
        lower = 10; upper = 100-lower
        thld_low = np.percentile(distances_norm,lower); thld_up = np.percentile(distances_norm,upper)
        low.append(thld_low); high.append(thld_up)
        distance_low = distances_norm[(distances_norm<=thld_low).values]; distance_up = distances_norm[(distances_norm>=thld_up).values]
        # calculate fwd returns
        # shift asset returns forward one month for ease of calculations
        shifted_assets_mom = delta_assets_mom.shift(periods=-1)
        # pick subset 
        shifted_assets_mom = shifted_assets_mom[shifted_assets_mom.index<=corrs.index[i]]
        # calculate forward returns
        fwd_returns_low = shifted_assets_mom[shifted_assets_mom.index.isin(distance_low.index)]
        fwd_returns_high = shifted_assets_mom[shifted_assets_mom.index.isin(distance_up.index)]
        # output descriptive of fwd returns
        desc_low = pd.concat([desc_low, fwd_returns_low.describe()], axis=1) 
        desc_high = pd.concat([desc_high, fwd_returns_high.describe()], axis=1)

    desc_low.columns = desc_high.columns = corrs.index.date

    low_df = pd.DataFrame(low, index=corrs.index)
    high_df = pd.DataFrame(high, index=corrs.index)

    significant_factors_names = pd.DataFrame(significant_factors_names,index=corrs.index)


In [ ]:
# Strategy
def ptf_analytics(total_returns: pd.DataFrame, returns: pd.DataFrame):
    num_yrs = (total_returns.index[-1] - total_returns.index[0])
    num_yrs = num_yrs.days/365
    # print(num_yrs)
    cagr = total_returns.iloc[-1]**(1/num_yrs)-1
    VaR = (1+np.percentile(returns,5))**(12)-1
    CVaR = returns[returns<np.percentile(returns,5)].mean()
    CVaR = (1+CVaR)**(12)-1
    ann_ret = (1+returns.mean())**12-1
    ann_vol = returns.std()*np.sqrt(12)
    cash_ret = (1+cash_mom.mean())**12-1
    sharpe = (ann_ret-cash_ret)/ann_vol
    returns_yearly = pd.DataFrame(returns.values,columns=['Long/Short'])
    returns_yearly["yr"] = corr_spx.index.year
    returns_grouped = (1+returns_yearly.groupby('yr').mean())**12-1
    t = [cagr, ann_ret, ann_vol,sharpe, CVaR, total_returns.iloc[-1]]
    id =  ["CAGR", "Ann. Ret", "Ann. Vol", "Sharpe","CVaR","Total Ret"]
    performance = pd.DataFrame(t, id)
    return performance, returns_grouped

threshold = desc_low.T[["mean"]]
r,c = threshold.shape
# shifted_assets_mom = delta_assets_mom.shift(periods=-1)
avg_fwd_ret = delta_assets_mom.shift(periods=-1).dropna()
avg_fwd_ret = avg_fwd_ret.iloc[-r:].squeeze()
# avg_fwd_ret = pd.DataFrame(avg_fwd_ret)
strategy_lc = []; strategy_ls = []
for i in range(r):
    if (threshold.iloc[i].values > 0):
        strategy_lc.append(avg_fwd_ret.iloc[i])
        strategy_ls.append(avg_fwd_ret.iloc[i])
    else:
        strategy_lc.append(cash_mom.iloc[i])
        strategy_ls.append(-avg_fwd_ret.iloc[i])

strategy_lc = pd.DataFrame(data=strategy_lc, index=desc_low.T.index, columns=[["Long/Cash"]]).dropna()
strategy_ls = pd.DataFrame(data=strategy_ls, index=desc_low.T.index, columns=[["Long/Short"]]).dropna()
strategy_lc_total_ret = (1 + strategy_lc).cumprod()
strategy_ls_total_ret = (1 + strategy_ls).cumprod()
assets_total_ret = (1+ avg_fwd_ret).cumprod()

strategy_lc_perf,strategy_lc_yearly = ptf_analytics(total_returns=strategy_lc_total_ret, returns=strategy_lc)
strategy_ls_perf,strategy_ls_yearly = ptf_analytics(strategy_ls_total_ret, strategy_ls)
assets_perf,asset_yearly = ptf_analytics(assets_total_ret, avg_fwd_ret)
# strategy_cagr = strategy[-1]
output2_strat = pd.concat([strategy_lc_perf, strategy_ls_perf, assets_perf],axis=1)
output2_strat.columns = [["Long/Cash", "Long/Short", delta_assets_mom.name]]
output_strat = pd.concat([strategy_ls.describe(), strategy_ls.describe(), pd.DataFrame(avg_fwd_ret.describe())],axis=1)

output_yearly=pd.concat([strategy_lc_yearly,strategy_ls_yearly,asset_yearly],axis=1)
output_yearly.columns=[["Long/Cash", "Long/Short", delta_assets_mom.name]]

In [15]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=strategy_lc_total_ret.index,y=strategy_lc_total_ret.values.flatten(), mode='lines',name="Long/Cash"))
fig.add_trace(go.Scatter(x=strategy_ls_total_ret.index,y=strategy_ls_total_ret.values.flatten(), mode='lines',name="Long/Short"))
fig.add_trace(go.Scatter(x=assets_total_ret.index,y=assets_total_ret.values.flatten(), mode='lines+markers',name="Long Only"))
fig.show()
output_strat
# cbr_strategy(trigger=desc_low, asset_return=delta_assets_mom)

NameError: name 'strategy_lc_total_ret' is not defined

In [ ]:
marker = datetime.datetime.now().strftime("%m-%d-%Y-%H.%M.%S")
with pd.ExcelWriter(data_assets.name+"_"+marker+'.xlsx') as writer:
    corr_spx.to_excel(writer, sheet_name="Correlations")
    desc_low.T.to_excel(writer, sheet_name="Return Stats")
    desc_high.T.to_excel(writer, sheet_name="Return Stats", startcol=12)
    distances_norm.to_excel(writer, sheet_name="Distances (Last)")
    distances_full_subset.to_excel(writer, sheet_name="Distances (Last)", startcol=4)
    delta_assets_mom.to_excel(writer, sheet_name = "data")
    delta_macro_yoy.to_excel(writer, sheet_name = "data", startcol=4)
    output2_strat.to_excel(writer, sheet_name = "PERF")
    strategy_lc_total_ret.to_excel(writer, sheet_name= "PERF", startcol=6)
    strategy_ls_total_ret.to_excel(writer, sheet_name= "PERF", startcol=9)
    assets_total_ret.to_excel(writer, sheet_name="PERF", startcol=12)
    output_yearly.to_excel(writer, sheet_name="PERF", startrow=12)
    significant_factors_names.to_excel(writer, sheet_name="PERF", startcol = 16)
# def mult_corr_pct(macro_pct: pd.DataFrame, corr: pd.DataFrame):
#     r_corr,c_corr = corr.shape
#     r_macro,c_macro = macro_pct.shape
#     # corr = corr[-min(r_corr,r_macro)]
#     # macro_pct = macro_pct[-min(r_corr,r_macro)]
#     factor_sel = np.abs(corr_spx) > 0.05
#     for i in range(c_corr):

#     return corr_pct_df